## Getting the data
We obtain the data using Google Big Query.
In order to obtain the deposits and withdrawals transactions, just run the following querys:

### Withdrawals 
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0x21a0adb6";

### Deposits
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0xb214faa5";

References here:
https://github.com/Phread420/tornado_bigquery/blob/main/Notes.md

This results are stored in the following files:
- withdraw_transactions.csv
- deposit_transactions.csv

After obtaining those two files, the next thing to do is to get the "recipient_addresses" that are making the withdrawl (if the user withdraws via a relayer, the relayer's account will be displayed in the "from_addres" field). That information is encoded in the "input" field of the withdrawal transactions.

The process to do so is in the following notebook:
https://github.com/lambdaclass/tornado_cash_anonymity_tool/blob/main/notebooks/complete_withdraw_data_set.ipynb

Note that the recipient addresses obtained in this df are in upper case and the addresses coming from the big query are in lower case, so you have to make the transformation when running the heuristic.

## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [1]:
using DataFrames
using CSV
using ProgressBars

In [2]:
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

In [3]:
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame)

non_relayer_withdraw_transactions_df = filter(row -> row.from_address == row.recipient_address, withdraw_transactions_df);

### Function summary: filter_by_unique_gas_price

Given a transactions DataFrame, it returns a dictionary with the unique gas prices as keys and a tuple containing the hash of that transaction and the timestamp. For example,

```
{111000302: ("0x387fcd87a", timestamp1)
```

In [4]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price.

function filter_by_unique_gas_price(transactions_df)
    unique_gas_prices = filter(row -> row.count==1, combine(groupby(transactions_df, :gas_price), nrow => :count))[!, "gas_price"]
    temp_df = filter(row -> row.gas_price ∈ unique_gas_prices, transactions_df)
    
    
    return Dict{Int, Tuple{String,String}}(
            (row.gas_price) => (row.hash, row.block_timestamp) for row in eachrow(temp_df)
            )

end

filter_by_unique_gas_price (generic function with 1 method)

### Function summary: filter_by_unique_gas_price_by_pool

Given a transactions DataFrame, it returns a dictionary with a tuple of the unique gas prices and tornado pool as keys and a tuple containing the hash of that transaction and the timestamp. For example, 

```
{(111000302, "0x24387248"): ("0x387fcd87a", timestamp1)

```

In [5]:
function filter_by_unique_gas_price_by_pool(transactions_df)
    unique_gas_prices_by_pool = filter(row -> row.count==1, combine(groupby(transactions_df, [:gas_price , :tornado_cash_address]), nrow => :count))[!, ["gas_price", "tornado_cash_address"]]
    
    
    # Tuple set with the values (gas_price, tornado_cash_address) is made to filter efficiently
    
    tuple_set = Set{Tuple{Int, String}}([(row.gas_price, row.tornado_cash_address) for row in eachrow(unique_gas_prices_by_pool)])
    
    # A dataframe of the transactions that verify having these unique gas price values by pool.
    
    temp_df = filter(iter_tuple -> (iter_tuple.gas_price, iter_tuple.tornado_cash_address) in tuple_set , deposit_transactions_df)
    
    # The DataFrame is transformed to a dictionary with only the relevant information.
    
    return Dict{Tuple{Int,String}, Tuple{String,String}}((row.gas_price, row.tornado_cash_address) => (row.hash, row.block_timestamp) for row in eachrow(temp_df))
end

filter_by_unique_gas_price_by_pool (generic function with 1 method)

### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a dictionary with the unique gas price deposits, as returned by the function filter_by_unique_gas_price.

It returns a tuple:
* $(True, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(False, None)$ when such a deposit is not found.

In [6]:
function same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dict)
    
    """
    Check if the gas price of the given withdrawal transaction is within the dictionary keys and that
    the deposit was made earlier than the withdral.
    If it is, then a tuple is returned, (True, deposit_hash).
    If it is not, (False, None) is returned.
    """
    
    if withdrawal_transaction.gas_price in Set(keys(unique_gas_price_deposit_dict)) && (unique_gas_price_deposit_dict[withdrawal_transaction.gas_price][2] < withdrawal_transaction.block_timestamp)
        return (true, unique_gas_price_deposit_dict[withdrawal_transaction.gas_price][1])
    else
        return (false, nothing)
    end
end

same_gas_price_heuristic (generic function with 1 method)

### Function summary: same_gas_price_heuristic_by_pool

This function receives a particular withdraw transaction and a dictionary with the unique gas price deposits made in each pool, just as returned by the function filter_by_unique_gas_price_by_pool.

It returns a tuple:
* $(True, deposit$ $hash)$ when a deposit transaction with the same gas price and belonging from the same pool (for example, 1ETH) as the withdrawal transaction is found.
* $(False, None)$ when such a deposit is not found.

In [7]:
function same_gas_price_heuristic_by_pool(withdrawal_transaction, unique_gas_price_deposit_dict)
    
    """
    Check if the pair (gas_price, tornado_pool) of the given withdrawal transaction is within the dictionary keys and that
    the deposit was made earlier than the withdral.
    If it is, then a tuple is returned, (True, deposit_hash).
    If it is not, (False, None) is returned.
    """
    
    if (withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address) in Set(keys(unique_gas_price_deposit_dict)) && (unique_gas_price_deposit_dict[(withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address)][2] < withdrawal_transaction.block_timestamp)
        return (true, unique_gas_price_deposit_dict[(withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address)][1])
    else  
        return (false, nothing)
    end
end

same_gas_price_heuristic_by_pool (generic function with 1 method)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [8]:
function apply_same_gas_price_heuristic(deposit_transactions_df, withdraw_transactions_df)
    
    # Get deposit transactions with unique gas prices.
    
    unique_gas_price_deposits_dict = filter_by_unique_gas_price(deposit_transactions_df)
    
    # Initialize an empty dictionary to store the linked transactions.
    
    withdrawal_to_deposit = Dict()
    
    # Iterate over the withdraw transactions.
    for withdraw_row in ProgressBar(eachrow(withdraw_transactions_df),  printing_delay=10)
        # Apply heuristic for the given withdraw transaction.
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits_dict)
        
        # When a deposit transaction matching the withdraw transaction gas price is found, add
        # the linked transactions to the dictionary.
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end

    # Return the linked transactions dictionary.
    return withdrawal_to_deposit
end

apply_same_gas_price_heuristic (generic function with 1 method)

### Function summary: apply_same_gas_price_heuristic_by_pool

Applies the heuristic to all the withdraw_transactions DataFrame, filtering also by the particular pool. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [9]:
function apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, withdraw_transactions_df)
    
    # Get deposit transactions with unique gas prices.
    
    unique_gas_price_deposits_dict = filter_by_unique_gas_price_by_pool(deposit_transactions_df)
    
    # Initialize an empty dictionary to store the linked transactions.
    
    withdrawal_to_deposit = Dict()
    
    # Iterate over the withdraw transactions.
    for withdraw_row in ProgressBar(eachrow(withdraw_transactions_df), printing_delay=10) 
        # Apply heuristic for the given withdraw transaction.
        same_gas_deposit_hash = same_gas_price_heuristic_by_pool(withdraw_row, unique_gas_price_deposits_dict)
        
        # When a deposit transaction matching the withdraw transaction gas price is found, add
        # the linked transactions to the dictionary.
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end

    # Return the linked transactions dictionary.
    return withdrawal_to_deposit
end

apply_same_gas_price_heuristic_by_pool (generic function with 1 method)

We run the heuristic for the entire withdraw dataset

In [10]:
linked_transactions = apply_same_gas_price_heuristic(deposit_transactions_df, non_relayer_withdraw_transactions_df)

0.0%┣                                     ┫ 0/17.0k [00:10<-47:-19:-30, -10s/it]
0.0%┣                                        ┫ 1/17.0k [00:10<Inf:Inf, InfGs/it]
100.0%┣████████████████████████████████████┫ 17.0k/17.0k [00:15<00:00, 1.2kit/s]


Dict{Any, Any} with 128 entries:
  "0x571a9c4a9d2a2b0386285e048073a8d48e995f2b5a3b5cd36734023f06e45a44" => "0x35a464d0e00fc398607b092f7b3186cd6c8ec0c82f6f4b6d9978c71221647671"
  "0x99b09453c62a24b9b7f78c27b37d1c06c2b9a64285412bf7e73609746a00abc0" => "0x1cf1ce4d853c7f7df4c44634e45080e6c038dcd0c4e651920138477d225cdd72"
  "0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd" => "0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f"
  "0x02a1c2250c4dea47f8f38f262ea723ff54de15e8ff6f4de9b19938757800868f" => "0x04e9849765f104c250e6da696b16fd2be7f0eca3254038c0ba21049b061b3748"
  "0x197b4399402095682fd725c4fa7ce23450f1ffb4bd651dbe4f992b231f61160e" => "0x988638a03002b0bdf7023fdc03c0e7f9fdb7a493a913dc5b5bfc7e5c3c26f3db"
  ⋮                                                                    => ⋮

In [11]:
linked_transactions = DataFrame([collect(keys(linked_transactions)), collect(values(linked_transactions))], ["withdraw_hashes","deposit_hashes"])

,withdraw_hashes,deposit_hashes
,Any,Any
1,0x571a9c4a9d2a2b0386285e048073a8d48e995f2b5a3b5cd36734023f06e45a44,0x35a464d0e00fc398607b092f7b3186cd6c8ec0c82f6f4b6d9978c71221647671
2,0x99b09453c62a24b9b7f78c27b37d1c06c2b9a64285412bf7e73609746a00abc0,0x1cf1ce4d853c7f7df4c44634e45080e6c038dcd0c4e651920138477d225cdd72
3,0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd,0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f
4,0x02a1c2250c4dea47f8f38f262ea723ff54de15e8ff6f4de9b19938757800868f,0x04e9849765f104c250e6da696b16fd2be7f0eca3254038c0ba21049b061b3748
5,0x197b4399402095682fd725c4fa7ce23450f1ffb4bd651dbe4f992b231f61160e,0x988638a03002b0bdf7023fdc03c0e7f9fdb7a493a913dc5b5bfc7e5c3c26f3db
6,0x3e7734771217c28f9fc6d5b412ab35b4f8554b22fa5bb4d59789a981d15cde34,0x425066c1edecb08e72b81b7e32cb01e9637cbd0e8442bc4ee06d071ce2690340
7,0x0ee6d471ad8bb4217ed0e416e2c51af2d6e486e0f3fe5ac187e0488add7b2a60,0x1cf1ce4d853c7f7df4c44634e45080e6c038dcd0c4e651920138477d225cdd72
8,0x2d9649c3437d75adf66be1e2aa0a20858fafc4bfdc9d591a643ae8c074e6fc64,0xb3df9977fcf74b589dac668cb4fb360732f76edb72d1a779c7e4ef57265d89cd
9,0x523ec14086e2788e60f3ec20bf57450a41175dd4d76990acf675199a145471d4,0x7f21ef508a9bfafb030071da5f98db43981d7ea5405fd327a46dafdb6f10b4c0


Now we run the heuristic that filters by pool, again applying it to the entire withdraw dataset.

In [12]:
linked_transactions_by_pool = apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, non_relayer_withdraw_transactions_df)

0.0%┣                                     ┫ 0/17.0k [00:10<-47:-19:-30, -10s/it]
0.0%┣                                        ┫ 1/17.0k [00:10<Inf:Inf, InfGs/it]
84.4%┣████████████████████████████████      ┫ 14.4k/17.0k [00:20<00:04, 718it/s]
100.0%┣█████████████████████████████████████┫ 17.0k/17.0k [00:22<00:00, 780it/s]


Dict{Any, Any} with 172 entries:
  "0x185124e22a1f913f3f71db70fc38d7ae56cd2e671bc08ebcd44d2de6f488cdbe" => "0xf72d23287b3bf7dfc53633e3be0f728d9d5068eadb927eb79045d31d34f6538d"
  "0xc1b92522a23530412f24c0edfe62be443d8d0c3217562ca184505249e0235bba" => "0x936d0599d352096cdfcc1df76c3a027ce3ff2d835cd0b3030e7aaa688ce2fdcb"
  "0x88fde9e123ac0a9b011428ffb713c424c8aa8c5980a23fb7d8f2ba157f50d29e" => "0x0e1295215085d3f9e4799e8fd2a70fba45ce15dad27e284b71a503a630f5f542"
  "0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd" => "0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f"
  "0x4ce400034e4118bb1f57613ab19bb3b4697e1a569abe34b50a4ee26d03af6bd0" => "0x536caff3cb304258ddd02281f9122b73e1a0e2f24a6e102e45ab293b2cdc14a1"
  ⋮                                                                    => ⋮

In [13]:
linked_transactions_by_pool = DataFrame([collect(keys(linked_transactions_by_pool)), collect(values(linked_transactions_by_pool))], ["withdraw_hashes","deposit_hashes"])

,withdraw_hashes,deposit_hashes
,Any,Any
1,0x185124e22a1f913f3f71db70fc38d7ae56cd2e671bc08ebcd44d2de6f488cdbe,0xf72d23287b3bf7dfc53633e3be0f728d9d5068eadb927eb79045d31d34f6538d
2,0xc1b92522a23530412f24c0edfe62be443d8d0c3217562ca184505249e0235bba,0x936d0599d352096cdfcc1df76c3a027ce3ff2d835cd0b3030e7aaa688ce2fdcb
3,0x88fde9e123ac0a9b011428ffb713c424c8aa8c5980a23fb7d8f2ba157f50d29e,0x0e1295215085d3f9e4799e8fd2a70fba45ce15dad27e284b71a503a630f5f542
4,0x062c1c415381703a6a00022bbb93ac541ca0092f84c8cc34658e4224da945bdd,0x42caf310513b6eed265b9cd7e260c0325282d8d209035188d7d74b3ef85f509f
5,0x4ce400034e4118bb1f57613ab19bb3b4697e1a569abe34b50a4ee26d03af6bd0,0x536caff3cb304258ddd02281f9122b73e1a0e2f24a6e102e45ab293b2cdc14a1
6,0xc4b5fc604fbddc059d9f734a6cd215da7a8396c78f3f40191081e2307db135f4,0x9da5cc9fd17e1f8f75a921dc0974fb7cf1d24077b5332a9630ca62c21e1263b3
7,0x4c5682d081e3085062c5f59bd29da9b80870fac27c94db1d4b58c443507dabbe,0xac60db623d36ab867f39572d07972b46809a7c7ceec8c852255bbb3730e0c5bf
8,0x0fae022876b43a8ffbf966b3135d935db856203d9862b1f560b9f75c9c2c2158,0xe70dd917d26dea5e9a668b1b6b87fef1baf5cdc0367e72211512a28fa51f7b04
9,0x1e84a2a314747ef8617b8ae62d0aa21058c8eab8e311d9100f376e88f870f13b,0x3e7dd307c469ded667789467b1feb3c9b263301ef862b833c01791944bf2f6a2
